In [1]:
from sklearn.ensemble import GradientBoostingClassifier
import pickle
from sklearn.metrics import ndcg_score
import pandas as pd
import numpy as np
import gc

In [2]:
# training set
x = pd.read_csv("data/dummy/train.csv")

# val set 20%
x_val = x.sample(n=int(0.1*len(x)))
x = x.drop(x_val.index)

## TODO: date_time
x.drop(['click_bool', 'booking_bool', 'date_time'], axis=1, inplace=True)
x_val.drop(['click_bool', 'booking_bool', 'date_time'], axis=1, inplace=True)

# sort both on search id
x.sort_values('srch_id', ignore_index=True)
x_val.sort_values('srch_id', ignore_index=True)

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,random_bool,agg_comp_rate,agg_comp_perc,agg_comp_inv,target
0,1,12,187,219,74474,3,4.5,1,2.40,0.1255,...,4,0,1,1,-1.00,1,0.000000,0.000000,0.00,0
1,1,12,187,219,88218,4,3.5,1,2.77,0.1266,...,4,0,1,1,-1.00,1,0.000000,0.000000,-0.25,0
2,1,12,187,219,89073,4,4.0,1,2.08,0.0150,...,4,0,1,1,-1.00,1,0.000000,5.000000,0.00,0
3,4,5,219,219,56063,4,4.5,0,3.26,0.3246,...,1,0,1,1,238.33,1,0.000000,0.000000,0.00,0
4,4,5,219,219,83045,3,3.5,0,3.26,0.3431,...,1,0,1,1,238.33,1,0.000000,0.000000,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495829,332782,19,158,158,26348,4,0.0,1,0.00,0.0000,...,1,0,1,1,-1.00,0,0.000000,0.000000,0.00,0
495830,332784,24,219,100,82051,3,3.5,0,4.17,0.0723,...,1,0,1,0,328.38,0,1.000000,23.666667,0.00,0
495831,332784,24,219,100,12976,3,4.5,0,4.84,0.0746,...,1,0,1,0,329.88,0,0.000000,0.000000,0.00,0
495832,332784,24,219,100,51585,4,4.5,0,4.67,0.0258,...,1,0,1,0,329.34,0,0.000000,0.000000,0.00,0


In [3]:
y_val = x_val.target.squeeze()
y = x.target.squeeze()
x.drop(['target'], axis=1, inplace=True)
x_val.drop(['target'], axis=1, inplace=True)

In [4]:
groups = x.srch_id.value_counts().sort_index()
groups_val = x_val.srch_id.value_counts().sort_index()

In [5]:
pd.set_option('display.max_columns', None)
x.head(n=1)

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,random_bool,agg_comp_rate,agg_comp_perc,agg_comp_inv
0,1,12,187,219,893,3,3.5,1,2.83,0.0438,4.95,104.77,0,23246,1,0,4,0,1,1,-1.0,1,0.0,0.0,0.0


In [6]:
categorical_columns = [1,2,3,4,7,13,19]
x.iloc[:,categorical_columns]

,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_brand_bool,srch_destination_id,srch_saturday_night_bool
0,12,187,219,893,1,23246,1
1,12,187,219,10404,1,23246,1
2,12,187,219,21315,1,23246,1
3,12,187,219,27348,1,23246,1
4,12,187,219,29604,1,23246,1
...,...,...,...,...,...,...,...
4958342,5,219,219,77700,1,16974,0
4958343,5,219,219,88083,1,16974,0
4958344,5,219,219,94508,1,16974,0
4958345,5,219,219,128360,1,16974,0


In [7]:
import lightgbm

# default lightgbm model with sklearn api
gbm = lightgbm.LGBMRanker(metric="ndcg", objective="lambdarank", num_boost_round=1000) 

#label_gain

##rank_xendcg, XE_NDCG_MART ranking objective function, aliases: xendcg, xe_ndcg, xe_ndcg_mart, xendcg_mart
#rank_xendcg is faster than and achieves the similar performance as lambdarank

# fitting model 
gbm.fit(
    x,
    y,
    eval_set = [(x_val, y_val)],
    group = groups,
    eval_group = [groups_val],
    verbose=1,
    early_stopping_rounds=100,
    categorical_feature=categorical_columns
)

/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [1, 2, 3, 4, 7, 13, 19]
  warnings.warn('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] num_iterations is set=1000, num_boost_round=1000 will be ignored. Current value: num_iterations=1000


/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[1]	valid_0's ndcg@1: 0.941591	valid_0's ndcg@2: 0.959689	valid_0's ndcg@3: 0.967867	valid_0's ndcg@4: 0.9713	valid_0's ndcg@5: 0.972658
Training until validation scores don't improve for 100 rounds
[2]	valid_0's ndcg@1: 0.943102	valid_0's ndcg@2: 0.96082	valid_0's ndcg@3: 0.968717	valid_0's ndcg@4: 0.972087	valid_0's ndcg@5: 0.973427
[3]	valid_0's ndcg@1: 0.943301	valid_0's ndcg@2: 0.960975	valid_0's ndcg@3: 0.968967	valid_0's ndcg@4: 0.972276	valid_0's ndcg@5: 0.973626
[4]	valid_0's ndcg@1: 0.94393	valid_0's ndcg@2: 0.961496	valid_0's ndcg@3: 0.969293	valid_0's ndcg@4: 0.972573	valid_0's ndcg@5: 0.97391
[5]	valid_0's ndcg@1: 0.944113	valid_0's ndcg@2: 0.961501	valid_0's ndcg@3: 0.969456	valid_0's ndcg@4: 0.97267	valid_0's ndcg@5: 0.973986
[6]	valid_0's ndcg@1: 0.944468	valid_0's ndcg@2: 0.961823	valid_0's ndcg@3: 0.969649	valid_0's ndcg@4: 0.972856	valid_0's ndcg@5: 0.974135
[7]	valid_0's ndcg@1: 0.944479	valid_0's ndcg@2: 0.961847	valid_0's ndcg@3: 0.969638	valid_0's ndcg@4: 0.97284

[60]	valid_0's ndcg@1: 0.945822	valid_0's ndcg@2: 0.963015	valid_0's ndcg@3: 0.970575	valid_0's ndcg@4: 0.973559	valid_0's ndcg@5: 0.974862
[61]	valid_0's ndcg@1: 0.945833	valid_0's ndcg@2: 0.96301	valid_0's ndcg@3: 0.97057	valid_0's ndcg@4: 0.973567	valid_0's ndcg@5: 0.974866
[62]	valid_0's ndcg@1: 0.94585	valid_0's ndcg@2: 0.963031	valid_0's ndcg@3: 0.9706	valid_0's ndcg@4: 0.973561	valid_0's ndcg@5: 0.974865
[63]	valid_0's ndcg@1: 0.945804	valid_0's ndcg@2: 0.963063	valid_0's ndcg@3: 0.970567	valid_0's ndcg@4: 0.973558	valid_0's ndcg@5: 0.974861
[64]	valid_0's ndcg@1: 0.945747	valid_0's ndcg@2: 0.963027	valid_0's ndcg@3: 0.970547	valid_0's ndcg@4: 0.973536	valid_0's ndcg@5: 0.974852
[65]	valid_0's ndcg@1: 0.945724	valid_0's ndcg@2: 0.963036	valid_0's ndcg@3: 0.970579	valid_0's ndcg@4: 0.973544	valid_0's ndcg@5: 0.974836
[66]	valid_0's ndcg@1: 0.94573	valid_0's ndcg@2: 0.963015	valid_0's ndcg@3: 0.970562	valid_0's ndcg@4: 0.97353	valid_0's ndcg@5: 0.974832
[67]	valid_0's ndcg@1: 0.94

[119]	valid_0's ndcg@1: 0.945952	valid_0's ndcg@2: 0.963229	valid_0's ndcg@3: 0.970692	valid_0's ndcg@4: 0.973711	valid_0's ndcg@5: 0.974972
[120]	valid_0's ndcg@1: 0.945953	valid_0's ndcg@2: 0.963227	valid_0's ndcg@3: 0.970713	valid_0's ndcg@4: 0.973717	valid_0's ndcg@5: 0.974978
[121]	valid_0's ndcg@1: 0.945958	valid_0's ndcg@2: 0.963237	valid_0's ndcg@3: 0.970716	valid_0's ndcg@4: 0.973725	valid_0's ndcg@5: 0.974981
[122]	valid_0's ndcg@1: 0.945993	valid_0's ndcg@2: 0.963235	valid_0's ndcg@3: 0.970754	valid_0's ndcg@4: 0.973739	valid_0's ndcg@5: 0.974995
[123]	valid_0's ndcg@1: 0.946016	valid_0's ndcg@2: 0.963265	valid_0's ndcg@3: 0.970749	valid_0's ndcg@4: 0.973749	valid_0's ndcg@5: 0.975003
[124]	valid_0's ndcg@1: 0.946027	valid_0's ndcg@2: 0.963246	valid_0's ndcg@3: 0.970749	valid_0's ndcg@4: 0.973746	valid_0's ndcg@5: 0.975
[125]	valid_0's ndcg@1: 0.94601	valid_0's ndcg@2: 0.963222	valid_0's ndcg@3: 0.970746	valid_0's ndcg@4: 0.973721	valid_0's ndcg@5: 0.974993
[126]	valid_0's n

[178]	valid_0's ndcg@1: 0.945834	valid_0's ndcg@2: 0.963193	valid_0's ndcg@3: 0.970659	valid_0's ndcg@4: 0.973674	valid_0's ndcg@5: 0.974918
[179]	valid_0's ndcg@1: 0.945833	valid_0's ndcg@2: 0.963214	valid_0's ndcg@3: 0.970664	valid_0's ndcg@4: 0.973675	valid_0's ndcg@5: 0.974923
[180]	valid_0's ndcg@1: 0.945856	valid_0's ndcg@2: 0.963239	valid_0's ndcg@3: 0.970673	valid_0's ndcg@4: 0.973674	valid_0's ndcg@5: 0.974935
[181]	valid_0's ndcg@1: 0.945908	valid_0's ndcg@2: 0.963255	valid_0's ndcg@3: 0.970688	valid_0's ndcg@4: 0.973697	valid_0's ndcg@5: 0.974955
[182]	valid_0's ndcg@1: 0.945936	valid_0's ndcg@2: 0.963264	valid_0's ndcg@3: 0.970698	valid_0's ndcg@4: 0.973704	valid_0's ndcg@5: 0.974963
[183]	valid_0's ndcg@1: 0.945971	valid_0's ndcg@2: 0.963228	valid_0's ndcg@3: 0.970694	valid_0's ndcg@4: 0.973714	valid_0's ndcg@5: 0.974963
[184]	valid_0's ndcg@1: 0.945965	valid_0's ndcg@2: 0.963245	valid_0's ndcg@3: 0.970708	valid_0's ndcg@4: 0.973715	valid_0's ndcg@5: 0.974968
[185]	valid_0

LGBMRanker(metric='ndcg', num_boost_round=1000, objective='lambdarank')

In [8]:
gc.collect()
with open('model/model', 'wb') as f:
    pickle.dump(gbm, f)

In [9]:
with open('model/model', 'rb') as f:
    model = pickle.load(f)

In [10]:
test = pd.read_csv("data/dummy/test.csv", index_col=False)
test.drop('date_time', axis=1, inplace=True)

In [11]:
predictions = model.predict(test)

In [12]:
len(predictions)

4959183

In [13]:
submission = test[['srch_id', 'prop_id']]
submission['pred'] = predictions

<ipython-input-13-8354dd1f716d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['pred'] = predictions


In [14]:
del test
gc.collect()

84

In [15]:
submission.head(n=100)

,srch_id,prop_id,pred
0,1,3180,0.377956
1,1,5543,0.913976
2,1,14142,-0.031670
3,1,22393,-0.201185
4,1,24194,0.107079
...,...,...,...
95,10,11203,-0.682473
96,10,23777,-0.940336
97,10,31281,1.029637
98,10,37420,-1.312507


In [16]:
submission = submission.sort_values(['srch_id', 'pred'], ascending=[True, False])

In [17]:
submission.head(n=20)

,srch_id,prop_id,pred
9,1,54937,0.976714
5,1,28181,0.974169
23,1,99484,0.963179
1,1,5543,0.913976
8,1,50162,0.506548
12,1,61934,0.495359
22,1,95031,0.418183
16,1,74045,0.384736
0,1,3180,0.377956
20,1,90385,0.372844


In [18]:
submission[['srch_id', 'prop_id']].to_csv('submission.csv', index=False)